In [2]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv 
import os 

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [4]:
# Create LLM 
llm = ChatOpenAI(model="gpt-4o-mini", api_key=OPENAI_API_KEY) 

In [7]:
# Create Tools 
def add(a: int, b: int) -> int:
    """Adds a and b."""
    return a + b


def multiply(a: int, b: int) -> int:
    """Multiplies a and b."""
    return a * b

def send_email(email_address: str) -> None: 
    print(f"Sending email to {email_address}")

tools = [add, multiply, send_email] 

In [8]:
# Bind to LLM 
llm_with_tools = llm.bind_tools(tools)

# Query 
query = "What is 3 * 12" 

llm_with_tools.invoke(query) 

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_zPlFwHHZxAQNENkacHisqOUQ', 'function': {'arguments': '{"a":3,"b":12}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 91, 'total_tokens': 108}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_ba606877f9', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-bfb48d29-4589-4649-97c8-dc72b1b99a0a-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_zPlFwHHZxAQNENkacHisqOUQ', 'type': 'tool_call'}], usage_metadata={'input_tokens': 91, 'output_tokens': 17, 'total_tokens': 108})

In [9]:
query = "Can you send an email?" 

llm_with_tools.invoke(query)

AIMessage(content='Yes, I can send an email. Please provide the email address you would like to send it to.', response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 90, 'total_tokens': 112}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_611b667b19', 'finish_reason': 'stop', 'logprobs': None}, id='run-3623ecca-0ec8-49d2-a3ee-ef288af0bf8a-0', usage_metadata={'input_tokens': 90, 'output_tokens': 22, 'total_tokens': 112})

In [12]:
query = "Send an email to 123@example.com" 

llm_with_tools.invoke(query).tool_calls

[{'name': 'send_email',
  'args': {'email_address': '123@example.com'},
  'id': 'call_TPnL8qda7AkP8VHe3vMTcIhC',
  'type': 'tool_call'}]

In [13]:
from langchain_core.messages import HumanMessage

query = "What is 3 * 12"

messages = [HumanMessage(query)]

ai_msg = llm_with_tools.invoke(messages)

print(ai_msg.tool_calls) 

messages.append(ai_msg)

[{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_vvnC6ckiEJfAKUqUx27uMknY', 'type': 'tool_call'}]


In [20]:
tool_call = ai_msg.tool_calls[0]
  
args = tool_call['args'] 
mapping = {
    "add": add, 
    "multiply": multiply
}

mapping[tool_call['name']](**args)

36